# The Sphere Revenue Build Using TM Alt Data

TODO:

- NEED HIST DATA FROM KEVIN
- Linear Regression Model for [X, Y, Z] -> Postcards Attendance
    - maybe weekday, time of day, ...
- Linear Regression Model for Postcards Attendance -> ASP
- Linear Regression Model for Postcards Attendance, [X, Y, Z] -> F&B Sales
    - weekday, time of day, ASP

In [101]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [102]:
df = pd.read_excel('sphr_visible_event_data_v1.xlsx')

df['Date & Time'] = pd.to_datetime(df['Date & Time'], utc=True)
# Extracting just the date part
df['Event Date'] = df['Date & Time'].dt.date

df.head()

,Group Name,Event Name,Date & Time,Average GA Ticket Price,Total Ticket Units Available,Aggregate Ticket Value,Event Date
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28 04:00:00+00:00,195.0,20000,3900000.0,2024-01-28
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18 04:00:00+00:00,195.0,20000,3900000.0,2024-02-18
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27 04:00:00+00:00,195.0,20000,3900000.0,2024-01-27
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04 04:00:00+00:00,195.0,20000,3900000.0,2024-02-04
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03 04:00:00+00:00,195.0,20000,3900000.0,2024-03-03


In [103]:
dfphish = df[df['Group Name'] == 'Phish']
dfphish.head()

,Group Name,Event Name,Date & Time,Average GA Ticket Price,Total Ticket Units Available,Aggregate Ticket Value,Event Date
43,Phish,Phish,2024-04-19 02:30:00+00:00,119.0,20000,2380000.0,2024-04-19
44,Phish,Phish,2024-04-21 02:30:00+00:00,119.0,20000,2380000.0,2024-04-21
45,Phish,Phish,2024-04-20 02:30:00+00:00,119.0,20000,2380000.0,2024-04-20
46,Phish,Phish,2024-04-22 02:30:00+00:00,119.0,20000,2380000.0,2024-04-22


In [104]:
# Establish Attendance Distribution

attendance_means = {
    'U2 Residence': 0.9,
    'Phish': 1,
    'Postcards': 0.7
}
attendance_std = {
    'U2 Residence': 0.05,
    'Phish': 0,
    'Postcards': 0.1
}

attendance_df = pd.DataFrame(index=df.index, columns=['Group Name', 'Event Name', 'Date & Time', 'Bear Case Attendance', 'Base Case Attendance', 'Bull Case Attendance'])
attendance_df['Group Name'] = df['Group Name']
attendance_df['Event Name'] = df['Event Name']
attendance_df['Date & Time'] = df['Date & Time']

for index, row in attendance_df.iterrows():
    group_name = row['Group Name']
    mean = attendance_means[group_name]
    std = attendance_std[group_name]
    attendance_df.loc[index, 'Bear Case Attendance'] = df.loc[index, 'Total Ticket Units Available'] * (mean - 2 * std)
    attendance_df.loc[index, 'Base Case Attendance'] = df.loc[index, 'Total Ticket Units Available'] * mean
    attendance_df.loc[index, 'Bull Case Attendance'] = df.loc[index, 'Total Ticket Units Available'] * (mean + 2 * std)

attendance_df.head()

,Group Name,Event Name,Date & Time,Bear Case Attendance,Base Case Attendance,Bull Case Attendance
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28 04:00:00+00:00,16000.0,18000.0,20000.0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18 04:00:00+00:00,16000.0,18000.0,20000.0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27 04:00:00+00:00,16000.0,18000.0,20000.0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04 04:00:00+00:00,16000.0,18000.0,20000.0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03 04:00:00+00:00,16000.0,18000.0,20000.0


In [105]:
asp = {
    'U2 Residence': 390,
    'Postcards': 100
}

ticket_revenue_df = attendance_df.copy()


for index, row in ticket_revenue_df.iterrows():
    ticket_revenue_df.loc[index, 'Bear Case Revenue'] = row['Bear Case Attendance'] * asp.get(row['Group Name'], df.loc[index, 'Average GA Ticket Price'])
    ticket_revenue_df.loc[index, 'Base Case Revenue'] = row['Base Case Attendance'] * asp.get(row['Group Name'], df.loc[index, 'Average GA Ticket Price'])
    ticket_revenue_df.loc[index, 'Bull Case Revenue'] = row['Bull Case Attendance'] * asp.get(row['Group Name'], df.loc[index, 'Average GA Ticket Price'])

ticket_revenue_df.drop(columns=['Bear Case Attendance', 'Base Case Attendance', 'Bull Case Attendance'], inplace=True)
ticket_revenue_df.head()

,Group Name,Event Name,Date & Time,Bear Case Revenue,Base Case Revenue,Bull Case Revenue
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28 04:00:00+00:00,6240000.0,7020000.0,7800000.0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18 04:00:00+00:00,6240000.0,7020000.0,7800000.0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27 04:00:00+00:00,6240000.0,7020000.0,7800000.0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04 04:00:00+00:00,6240000.0,7020000.0,7800000.0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03 04:00:00+00:00,6240000.0,7020000.0,7800000.0


# Bear Case Ticket Revenue Gives 95% Confidence Interval of Ticket Revs

In [106]:
MEAN_CONCERT_FNB_PER_CAPITA = 0.5
MEAN_POSTCARD_FNB_PER_CAPITA = 0.8
STD_FNB_PER_CAPITA = 0.1
MEAN_FNB_BILL = 35

fnb_df = attendance_df.copy()
fnb_df['Bear Case F&B Revenue'] = 0
fnb_df['Base Case F&B Revenue'] = 0
fnb_df['Bull Case F&B Revenue'] = 0
for index, row in fnb_df.iterrows():
    if row['Group Name'] == 'Postcards':
        fnb_df.loc[index, 'Bear Case F&B Revenue'] += row['Bear Case Attendance'] * (MEAN_POSTCARD_FNB_PER_CAPITA - 2 * STD_FNB_PER_CAPITA) * MEAN_FNB_BILL
        fnb_df.loc[index, 'Base Case F&B Revenue'] += row['Base Case Attendance'] * MEAN_POSTCARD_FNB_PER_CAPITA * MEAN_FNB_BILL
        fnb_df.loc[index, 'Bull Case F&B Revenue'] += row['Bull Case Attendance'] * 0.9 * MEAN_FNB_BILL
    else:
        fnb_df.loc[index, 'Bear Case F&B Revenue'] += row['Bear Case Attendance'] * (MEAN_CONCERT_FNB_PER_CAPITA - 2 * STD_FNB_PER_CAPITA) * MEAN_FNB_BILL
        fnb_df.loc[index, 'Base Case F&B Revenue'] += row['Base Case Attendance'] * MEAN_CONCERT_FNB_PER_CAPITA * MEAN_FNB_BILL
        fnb_df.loc[index, 'Bull Case F&B Revenue'] += row['Bull Case Attendance'] * 0.7 * MEAN_FNB_BILL

fnb_df.drop(columns=['Bear Case Attendance', 'Base Case Attendance', 'Bull Case Attendance'], inplace=True)
fnb_df.head()

,Group Name,Event Name,Date & Time,Bear Case F&B Revenue,Base Case F&B Revenue,Bull Case F&B Revenue
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28 04:00:00+00:00,168000,315000,490000
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18 04:00:00+00:00,168000,315000,490000
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27 04:00:00+00:00,168000,315000,490000
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04 04:00:00+00:00,168000,315000,490000
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03 04:00:00+00:00,168000,315000,490000


In [107]:
SPHR_TAKE_RATE = 0.1

sphr_revenue = pd.DataFrame(index = df.index, columns=['Group Name', 'Event Name', 'Date & Time', 'Bear Case Ticket Revenue', 'Base Case Ticket Revenue', 'Bull Case Ticket Revenue', 'Bear Case F&B Revenue', 'Base Case F&B Revenue', 'Bull Case F&B Revenue', 'Bear Case Total Revenue', 'Base Case Total Revenue', 'Bull Case Total Revenue'])

sphr_revenue['Group Name'] = df['Group Name']
sphr_revenue['Event Name'] = df['Event Name']
sphr_revenue['Date & Time'] = df['Date & Time']
# tickets
for index, row in sphr_revenue.iterrows():
    if row['Group Name'] == 'Postcards':
        sphr_revenue.loc[index, 'Bear Case Ticket Revenue'] = ticket_revenue_df.loc[index, 'Bear Case Revenue']
        sphr_revenue.loc[index, 'Base Case Ticket Revenue'] = ticket_revenue_df.loc[index, 'Base Case Revenue']
        sphr_revenue.loc[index, 'Bull Case Ticket Revenue'] = ticket_revenue_df.loc[index, 'Bull Case Revenue']
    else:
        sphr_revenue.loc[index, 'Bear Case Ticket Revenue'] = ticket_revenue_df.loc[index, 'Bear Case Revenue'] * SPHR_TAKE_RATE
        sphr_revenue.loc[index, 'Base Case Ticket Revenue'] = ticket_revenue_df.loc[index, 'Base Case Revenue'] * SPHR_TAKE_RATE
        sphr_revenue.loc[index, 'Bull Case Ticket Revenue'] = ticket_revenue_df.loc[index, 'Bull Case Revenue'] * SPHR_TAKE_RATE

# fnb
sphr_revenue['Bear Case F&B Revenue'] = fnb_df['Bear Case F&B Revenue']
sphr_revenue['Base Case F&B Revenue'] = fnb_df['Base Case F&B Revenue']
sphr_revenue['Bull Case F&B Revenue'] = fnb_df['Bull Case F&B Revenue']

# total
sphr_revenue['Bear Case Total Revenue'] = sphr_revenue['Bear Case Ticket Revenue'] + sphr_revenue['Bear Case F&B Revenue']
sphr_revenue['Base Case Total Revenue'] = sphr_revenue['Base Case Ticket Revenue'] + sphr_revenue['Base Case F&B Revenue']
sphr_revenue['Bull Case Total Revenue'] = sphr_revenue['Bull Case Ticket Revenue'] + sphr_revenue['Bull Case F&B Revenue']

# sort by Date & Time
sphr_revenue.sort_values(by=['Date & Time'], inplace=True)
sphr_revenue.head()

,Group Name,Event Name,Date & Time,Bear Case Ticket Revenue,Base Case Ticket Revenue,Bull Case Ticket Revenue,Bear Case F&B Revenue,Base Case F&B Revenue,Bull Case F&B Revenue,Bear Case Total Revenue,Base Case Total Revenue,Bull Case Total Revenue
42,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-05 22:00:00+00:00,250000.0,350000.0,450000.0,52500,98000,141750,302500.0,448000.0,591750.0
48,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06 00:30:00+00:00,250000.0,350000.0,450000.0,52500,98000,141750,302500.0,448000.0,591750.0
49,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06 03:00:00+00:00,250000.0,350000.0,450000.0,52500,98000,141750,302500.0,448000.0,591750.0
61,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06 05:30:00+00:00,250000.0,350000.0,450000.0,52500,98000,141750,302500.0,448000.0,591750.0
47,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06 22:00:00+00:00,250000.0,350000.0,450000.0,52500,98000,141750,302500.0,448000.0,591750.0


In [108]:
# export to excel
# convert Date & Time to datetime with no tz
sphr_revenue['Date & Time'] = sphr_revenue['Date & Time'].dt.tz_convert(None)
sphr_revenue.to_excel('sphr_revenue.xlsx', index=False)

# DViz

In [109]:
# Make a date column

grouped_revenues = pd.DataFrame(index = sphr_revenue.index, columns=['Date', 'Bear Case TR', 'Base Case TR', 'Bull Case TR'])
grouped_revenues['Date'] = sphr_revenue['Date & Time'].dt.date
grouped_revenues.drop_duplicates(subset=['Date'], inplace=True)
for index, row in grouped_revenues.iterrows():
    date = row['Date']
    grouped_revenues.loc[index, 'Bear Case TR'] = sphr_revenue[sphr_revenue['Date & Time'].dt.date == date]['Bear Case Total Revenue'].sum()
    grouped_revenues.loc[index, 'Base Case TR'] = sphr_revenue[sphr_revenue['Date & Time'].dt.date == date]['Base Case Total Revenue'].sum()
    grouped_revenues.loc[index, 'Bull Case TR'] = sphr_revenue[sphr_revenue['Date & Time'].dt.date == date]['Bull Case Total Revenue'].sum()

# Plot Bar Chart with the bars beside eachother
fig = go.Figure()
fig.add_trace(go.Bar(x=grouped_revenues['Date'], y=grouped_revenues['Bear Case TR'], name='Bear Case TR'))
fig.add_trace(go.Bar(x=grouped_revenues['Date'], y=grouped_revenues['Base Case TR'], name='Base Case TR'))
fig.add_trace(go.Bar(x=grouped_revenues['Date'], y=grouped_revenues['Bull Case TR'], name='Bull Case TR'))
fig.update_layout(title_text='SPHR Case Revenue By Day', xaxis_title='Date', yaxis_title='Revenue ($)', barmode='group', xaxis_tickangle=-45)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [110]:
# Plot Case Cumulative Revenue over Time

fig = go.Figure()
fig.add_trace(go.Scatter(x=grouped_revenues['Date'], y=grouped_revenues['Bear Case TR'].cumsum(), name='Bear Case TR'))
fig.add_trace(go.Scatter(x=grouped_revenues['Date'], y=grouped_revenues['Base Case TR'].cumsum(), name='Base Case TR'))
fig.add_trace(go.Scatter(x=grouped_revenues['Date'], y=grouped_revenues['Bull Case TR'].cumsum(), name='Bull Case TR'))
fig.update_layout(title_text='SPHR Case Cumulative Revenue', xaxis_title='Date', yaxis_title='Cumulative Revenue ($)')
fig.show()

In [111]:
ebit_margins = {
    'residency': 0.5,
    'postcards': 0.75,
    'food': 0.25
}

# Calculate EBIT
sphr_ebit = pd.DataFrame(index=sphr_revenue.index, columns=['Date', 'Ticket EBIT', 'F&B EBIT', 'Total EBIT'])
sphr_ebit['Date'] = sphr_revenue['Date & Time'].dt.date
sphr_ebit.drop_duplicates(subset=['Date'], inplace=True)
sphr_ebit['Ticket EBIT'] = 0
sphr_ebit['F&B EBIT'] = 0
sphr_ebit['Total EBIT'] = 0

for index, row in sphr_revenue.iterrows():
    if row['Group Name'] == 'Postcards':
        em = ebit_margins['postcards']
    else:
        em = ebit_margins['residency']
    date = row['Date & Time'].date()
    sphr_ebit.loc[sphr_ebit['Date'] == date, 'Ticket EBIT'] += row['Bear Case Ticket Revenue'] * em
    sphr_ebit.loc[sphr_ebit['Date'] == date, 'F&B EBIT'] += row['Bear Case F&B Revenue'] * ebit_margins['food']
    sphr_ebit.loc[sphr_ebit['Date'] == date, 'Total EBIT'] = sphr_ebit.loc[sphr_ebit['Date'] == date, 'Ticket EBIT'] + sphr_ebit.loc[sphr_ebit['Date'] == date, 'F&B EBIT']

# Plot EBIT over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=sphr_ebit['Date'], y=sphr_ebit['Ticket EBIT'].cumsum(), name='Ticket EBIT'))
fig.add_trace(go.Scatter(x=sphr_ebit['Date'], y=sphr_ebit['F&B EBIT'].cumsum(), name='F&B EBIT'))
fig.add_trace(go.Scatter(x=sphr_ebit['Date'], y=sphr_ebit['Total EBIT'].cumsum(), name='Total EBIT'))
fig.update_layout(title_text='SPHR EBIT', xaxis_title='Date', yaxis_title='EBIT ($)')
fig.show()